##### Flower project / Classification problem

<img src="./favicon.jpg" alt="flower image" />

###### Environment initialization

In [24]:
%tensorflow_version 2.x

UsageError: Line magic function `%tensorflow_version` not found.


In [25]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import pandas as pd

from IPython.display import clear_output


###### Dataset loading

In [26]:
#train_path = tf.keras.utils.get_file("iris_training.csv", "https://storage.googleapi.com/download.tensorflow.org/data/iris_training.csv")
#eval_path  = tf.keras.utils.get_file("iris_test.csv", "https://storage.googleapi.com/download.tensorflow.org/data/iris_test.csv")

CSV_COLUMN_NAME = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

df_train = pd.read_csv('iris_training.csv', names=CSV_COLUMN_NAME, header=0)
df_eval  = pd.read_csv('iris_test.csv', names=CSV_COLUMN_NAME, header=0)


In [27]:
df_train.head(5)

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [28]:
df_eval.head(5)

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,5.9,3.0,4.2,1.5,1
1,6.9,3.1,5.4,2.1,2
2,5.1,3.3,1.7,0.5,0
3,6.0,3.4,4.5,1.6,1
4,5.5,2.5,4.0,1.3,1


###### Dataset processing

In [29]:
y_train = df_train.pop('Species')
y_eval  = df_eval.pop('Species')

features_column = []
for key in df_train.keys():
    features_column.append(tf.feature_column.numeric_column(key=key))

print(features_column)    

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


###### convert pandas dataset to tf.data.Dataset Object

In [30]:
def input_fn(features, labels, training=True, batch_size=256):
    # let convert input to tensforflow Dataset fromat
    ds = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    
    # Shuffle if in training mode
    if training:
        ds = ds.shuffle(1000).repeat()
        
    return ds.batch(batch_size)

###### creating the model

In [31]:
classifier = tf.estimator.DNNClassifier(feature_columns=features_column, hidden_units=[30, 10], n_classes=3)
clear_output()

###### training the model

In [34]:
classifier.train(input_fn=lambda: input_fn(df_train, y_train, training=True), steps=5000)

# eval
eval_result = classifier.evaluate(input_fn=lambda: input_fn(df_eval, y_eval, training=False))
print('='*115)
print('\nTest set accuracy : {accuracy:0.3f}\n'.format(**eval_result))
print('='*115)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpzc6z1kgy/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 10000...
INFO:tensorflow:Saving checkpoints for 10000 into /tmp/tmpzc6z1kgy/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 10000...
INFO:tensorflow:loss = 0.394408, step = 10000
INFO:tensorflow:global_step/sec: 146.266
INFO:tensorflow:loss = 0.4087716, step = 10100 (0.687 sec)
INFO:tensorflow:global_step/sec: 169.964
INFO:tensorflow:loss = 0.38967985, step = 10200 (0.594 sec)
INFO:tensorflow:global_step/sec: 170.497
INFO:tensorflow:loss = 0.39698026, step = 10300 (0.581 sec)
INFO:tensorflow:global_step/sec: 175.393
INFO:tensorflow:loss = 0.37991345, step = 10400 (0.571 sec)
IN

###### make prediction

In [35]:
def prediction():
    def input_fn(features, batch_size=256):
        return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)
    
    features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
    predict = {}
    
    print("Please type numeric value as prompted:")
    for feature in features:
        valid = True
        while valid:
            val = input(feature + ": ")
            if not val.isdigit(): valid = False
            
        predict[feature] = [float(val)]
        
    predictions = classifier.predict(input_fn=lambda: input_fn(predict))
    for prediction in predictions:
        print(prediction)
        class_id = prediction['class_ids'][0]
        probability = prediction['probabilities'][class_id]
        
        print('{}\nPrediction is "{}" ({:.1f}%)\n{}'.format('='*115, SPECIES[class_id], 100 * probability, '='*115))

In [36]:
prediction()

Please type numeric value as prompted:
SepalLength: 5.1
SepalWidth: 3.3
PetalLength: 1.7
PetalWidth: 0.5
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpzc6z1kgy/model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'logits': array([-1.2299173, -3.4614465, -6.6655507], dtype=float32), 'probabilities': array([0.8995049 , 0.09657455, 0.00392047], dtype=float32), 'class_ids': array([0]), 'classes': array([b'0'], dtype=object), 'all_class_ids': array([0, 1, 2], dtype=int32), 'all_classes': array([b'0', b'1', b'2'], dtype=object)}
Prediction is "Setosa" (90.0%)


In [37]:
prediction()

Please type numeric value as prompted:
SepalLength: 5.9
SepalWidth: 3.0
PetalLength: 4.2
PetalWidth: 1.5
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpzc6z1kgy/model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'logits': array([-0.9289289,  1.245981 ,  0.8853576], dtype=float32), 'probabilities': array([0.06274278, 0.5522238 , 0.38503337], dtype=float32), 'class_ids': array([1]), 'classes': array([b'1'], dtype=object), 'all_class_ids': array([0, 1, 2], dtype=int32), 'all_classes': array([b'0', b'1', b'2'], dtype=object)}
Prediction is "Versicolor" (55.2%)


In [38]:
prediction()

Please type numeric value as prompted:
SepalLength: 6.9
SepalWidth: 3.1
PetalLength: 5.4
PetalWidth: 2.1
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpzc6z1kgy/model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'logits': array([-0.9672949,  2.7412453,  3.4153397], dtype=float32), 'probabilities': array([0.00820729, 0.33481002, 0.6569827 ], dtype=float32), 'class_ids': array([2]), 'classes': array([b'2'], dtype=object), 'all_class_ids': array([0, 1, 2], dtype=int32), 'all_classes': array([b'0', b'1', b'2'], dtype=object)}
Prediction is "Virginica" (65.7%)


In [ ]:
#---------------------------------------------
#here wehave an test case with espected result
#| SPECIES     | SL   SW   PL   PW  |
#"Sotosa"     => 5.1  3.3  1.7  0.5 
#"Versicolor" => 5.9  3.0  4.2  1.5
#"Virginica"  => 6.9  3.1  5.4  2.1
#---------------------------------------------